<a href="https://colab.research.google.com/github/jsqzhu/nbme/blob/main/02_NSMB_Bert_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 111.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 28.3 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as K
from tensorflow.keras import mixed_precision
#import tensorflow_addons as tfa
#import dill
import matplotlib.pyplot as plt
import transformers
from transformers import AutoTokenizer, AutoConfig,TFAutoModel
import json
import matplotlib.pyplot as plt
from ast import literal_eval 

In [3]:
train = pd.read_csv('https://raw.githubusercontent.com/jsqzhu/nbme/main/data.csv')
train.head()

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text,annot_count,loc_count
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-MI-OR-Family-history-of-myoc...,1,1
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-thyroid-disorder,1,1
2,00016_002,0,16,2,['chest pressure'],['203 217'],HPI: 17yo M presents with palpitations. Patien...,Chest-pressure,1,1
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms,2,2
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258'],HPI: 17yo M presents with palpitations. Patien...,Lightheaded,1,1


In [4]:
MODEL_NAME = "roberta-base"

## Resources

https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT

# Functions

In [5]:
def extract_location (location):
  """get start_char, end_char from location column"""
  #annotation = literal_eval(annotation)
  location = literal_eval(location)
  #annotation_list = []
  start_char = []
  end_char = []
  #if len(location)==0:
  #  start_char.append(-1)
  #  end_char.append(-1)
  #else:
  for n in range(len(location)):
      loc = location[n].split(' ')
      start_char.append(int(loc[0]))
      end_char.append(int(loc[1]))
  return start_char, end_char

# Prepare Data for Modeling

## Prepare feature entities

In [25]:
print('# unique feature labels: ', train['feature_num'].nunique())

# unique feature labels:  143


In [26]:
train['feature_num'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
       100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 300, 301, 302, 303, 304, 305, 306, 307, 308,
       309, 310, 311, 312, 314, 315, 313, 401, 402, 403, 404, 406, 407,
       408, 409, 405, 400, 500, 501, 502, 503, 504, 505, 508, 509, 510,
       511, 512, 513, 515, 516, 517, 506, 507, 514, 600, 601, 602, 603,
       604, 605, 606, 607, 609, 611, 608, 610, 700, 701, 702, 703, 704,
       705, 706, 707, 708, 800, 801, 802, 803, 804, 805, 806, 807, 808,
       809, 811, 812, 814, 815, 817, 813, 816, 810, 901, 902, 903, 904,
       905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 900])

Recall that 4399 cases have no corresponding text in pn_history for the target feature entity. Thus, we need to create an additional class to account for the texts with no annotations

In [27]:
Label = ["O"] + train['feature_num'].unique().tolist() 
le = LabelEncoder()
le.fit(Label)

LabelEncoder()

In [28]:
len(Label)

144

In [29]:
train['Target'] = le.transform(train['feature_num'])

In [30]:
EMPTY_idx = le.transform(["O"])[0]

In [31]:
EMPTY_idx

143

## Prepare dataset

Use transformer with TF2.0: https://www.kdnuggets.com/2021/02/hugging-face-transformer-basics.html#:~:text=NLP%2Dfocused%20startup%20Hugging%20Face,for%20training%20and%20evaluation%20purposes.

In [32]:
# load model
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
test_text = train['pn_history'][0]

https://huggingface.co/docs/transformers/main_classes/tokenizer

In [34]:
encodings = tokenizer(test_text, 
                           max_length=512,
                           padding='max_length',#sentence max to save on computation
                           truncation=True,
                           return_offsets_mapping=True)

In [35]:
print(f'text: \'{test_text}\'')
print(f'input ids: {encodings["input_ids"]}')
print(f'attention mask: {encodings["attention_mask"]}')
print(f'offset_mapping: {encodings["offset_mapping"]}')

text: 'HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. 
PMHx: none
Rx: uses friends adderrall
FHx: mom with "thyroid disease," dad with recent heart attcak
All: none
Immunizations: up to date
SHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms'
input ids: [0, 725, 11337, 35, 601, 9839, 256, 6822, 19, 87

In [36]:
for token_id, pos in zip(encodings['input_ids'], encodings['offset_mapping']):
    print(token_id, pos, test_text[pos[0]:pos[1]])

0 (0, 0) 
725 (0, 1) H
11337 (1, 3) PI
35 (3, 4) :
601 (5, 7) 17
9839 (7, 9) yo
256 (10, 11) M
6822 (12, 20) presents
19 (21, 25) with
8750 (26, 29) pal
17291 (29, 32) pit
1635 (32, 38) ations
4 (38, 39) .
27690 (40, 47) Patient
690 (48, 55) reports
155 (56, 57) 3
12 (57, 58) -
306 (58, 59) 4
377 (60, 66) months
9 (67, 69) of
33073 (70, 82) intermittent
7585 (83, 91) episodes
9 (92, 94) of
22 (95, 96) "
12690 (96, 101) heart
4108 (102, 109) beating
73 (109, 110) /
642 (110, 111) p
10773 (111, 118) ounding
66 (119, 122) out
9 (123, 125) of
127 (126, 128) my
7050 (129, 134) chest
72 (134, 136) ."
132 (137, 138) 2
360 (139, 143) days
536 (144, 147) ago
148 (148, 154) during
10 (155, 156) a
4191 (157, 163) soccer
177 (164, 168) game
56 (169, 172) had
41 (173, 175) an
3238 (176, 183) episode
6 (183, 184) ,
53 (185, 188) but
42 (189, 193) this
86 (194, 198) time
56 (199, 202) had
7050 (203, 208) chest
1164 (209, 217) pressure
8 (218, 221) and
1299 (222, 226) felt
25 (227, 229) as
114 (230, 2

In [37]:
# need to check the max length
# check if len(subtoken) longer than max_length from the model?
#https://ai.stackexchange.com/questions/37624/why-do-transformers-have-a-fixed-input-length
# how to handle longer tokens
#https://stackoverflow.com/questions/74290497/how-to-handle-sequences-longer-than-512-tokens-in-layoutlmv3
# for offset_mapping
# https://huggingface.co/transformers/v4.2.2/custom_datasets.html

use model tokenizer to prepare input text; 
convert pn_history to word embedding as input features 

In [38]:
train_copy = train.copy()

In [39]:
train_copy[train_copy['location'].str.contains(';')].head()

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text,annot_count,loc_count,Target,start_char,end_char


In [40]:
# remove records with error in annotation/location; will investigate later
train = train[~train['location'].str.contains(';')]

In [41]:
train['start_char'], train['end_char'] = zip(*train['location'].apply(lambda x: extract_location(x)))

In [42]:
train = train.explode(['start_char','end_char'])

In [43]:
train.head()

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text,annot_count,loc_count,Target,start_char,end_char
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-MI-OR-Family-history-of-myoc...,1,1,0,696,724
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-thyroid-disorder,1,1,1,668,693
2,00016_002,0,16,2,['chest pressure'],['203 217'],HPI: 17yo M presents with palpitations. Patien...,Chest-pressure,1,1,18,203,217
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms,2,2,36,70,91
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms,2,2,36,176,183


## SEQUENCE Length

In [44]:
pn_history=[]
input_id_sequence = []
attn_sequence = []
Target_labels = []
max_sequence_length = 0
for pn in train.groupby('pn_num'):
  # for loop to get encoding for each unique clinical note
  # output is a tuple (pn_num, df)
  df = pn[1] 
  #clinical note to be annotated
  pn_history = df.iloc[0].pn_history
  
  # get encoding
  encoding = tokenizer(pn_history, 
                       #max_length=512,
                       padding='longest', #'max_length',#sentence max to save on computation
                       truncation=False,
                       return_offsets_mapping=True)
  input_id = encoding['input_ids']
  attn = encoding['attention_mask']
  offset = encoding['offset_mapping']

  sequence_length = len(input_id)
  if sequence_length > max_sequence_length:
    max_sequence_length = sequence_length
  else:
    pass

print('max_sequence_length: ', max_sequence_length)

max_sequence_length:  392


## Testing target labeling

In [45]:
test = train[train.pn_num==16]

In [46]:
test

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text,annot_count,loc_count,Target,start_char,end_char
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-MI-OR-Family-history-of-myoc...,1,1,0,696,724
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-thyroid-disorder,1,1,1,668,693
2,00016_002,0,16,2,['chest pressure'],['203 217'],HPI: 17yo M presents with palpitations. Patien...,Chest-pressure,1,1,18,203,217
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms,2,2,36,70,91
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms,2,2,36,176,183
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms,2,2,36,70,91
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms,2,2,36,176,183
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258'],HPI: 17yo M presents with palpitations. Patien...,Lightheaded,1,1,53,222,258
5,00016_005,0,16,5,[],[],HPI: 17yo M presents with palpitations. Patien...,No-hair-changes-OR-no-nail-changes-OR-no-tempe...,0,0,64,NaN,NaN
6,00016_006,0,16,6,"['adderall', 'adderrall', 'adderrall']","['321 329', '404 413', '652 661']",HPI: 17yo M presents with palpitations. Patien...,Adderall-use,3,3,83,321,329


https://www.lighttag.io/blog/sequence-labeling-with-transformers/example

In [47]:
# create a dictionary of target and start_char, and target and end_char
#target_start_dict = {k:[j for i in v for j in i] for k, v in test.groupby('target')['start_char']}
#target_end_dict = {k:[j for i in v for j in i] for k, v in test.groupby('target')['end_char']}

In [48]:
# tokenize the text 
encoding = tokenizer(test['pn_history'][0], 
                       #max_length='512',
                       padding='longest',#sentence max to save on computation
                      truncation=False,
                        return_offsets_mapping=True)
input_id = encoding['input_ids']
attn = encoding['attention_mask']
offset = encoding['offset_mapping']

In [49]:
len(offset)

246

In [50]:
# create a list same length as tokens to store target labels
Target_entity_labels = []
Target_entity_labels = [EMPTY_idx]*len(offset)
# iterate through rows in dataframe to compare start/end char with the offset mapping position
for idx, row in test.iterrows(): 
  # compare
  start = row.start_char
  end = row.end_char
  target = row.Target
  for i, (pos_1, pos_2) in enumerate(offset):
    if (pos_1 >= start) and (pos_2 <= end): #
      Target_entity_labels[i] = target
    else:
      pass
#print(Target_entity_labels)

In [51]:
Target_entity_labels

[143,
 143,
 143,
 143,
 13,
 13,
 17,
 143,
 143,
 125,
 125,
 125,
 143,
 143,
 143,
 2,
 2,
 2,
 2,
 2,
 36,
 36,
 143,
 143,
 125,
 125,
 125,
 125,
 125,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 36,
 143,
 143,
 143,
 143,
 143,
 18,
 18,
 143,
 53,
 53,
 53,
 53,
 53,
 53,
 53,
 53,
 53,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 83,
 83,
 83,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 83,
 83,
 83,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 143,
 83,
 83,
 83,
 143,
 143,
 143,
 143,
 143

## Prepare data for Bert

In [52]:
pn_history=[]
input_id_sequence = []
attn_sequence = []
Target_labels = []
for pn in train.groupby('pn_num'):
  # for loop to get encoding for each unique clinical note
  # output is a tuple (pn_num, df)
  df = pn[1] 
  #clinical note to be annotated
  pn_history = df.iloc[0].pn_history
  
  # get encoding
  encoding = tokenizer(pn_history, 
                       max_length=max_sequence_length,
                       padding='max_length',#sentence max to save on computation
                       truncation=False,
                       return_offsets_mapping=True)
  input_id = encoding['input_ids']
  attn = encoding['attention_mask']
  offset = encoding['offset_mapping']

  # prepare target sequence
  # First, create a target_entity_sequence same length as encoding and fill by default 'O'
  Target_label = [EMPTY_idx]*len(offset)
  # Next, iterate through rows in df to compare start/end char with the offset_mapping position
  for idx, row in df.iterrows(): 
    start = row.start_char
    end = row.end_char
    target = row.Target

    for i, (pos_1, pos_2) in enumerate(offset):
      if (pos_1 >= start) and (pos_2 <= end): #
        Target_label[i] = target
      else:
        pass
  Target_labels.append(Target_label)
  input_id_sequence.append(input_id)
  attn_sequence.append(attn)


In [53]:
input_id_sequence = np.array(input_id_sequence).astype(np.int32)
attn_sequence = np.array(attn_sequence).astype(np.int32)
# the target class for each token will be one-hot encoded by an array [,144]
Target_labels = np.array(tf.keras.utils.to_categorical(Target_labels,len(Label))).astype(np.uint8)

In [54]:
Target_labels.shape

(1000, 392, 144)

In [55]:
print(len(input_id_sequence))
print(len(attn_sequence))
print(len(Target_labels))

1000
1000
1000


In [56]:
print(len(input_id_sequence[0]))
print(len(attn_sequence[0]))
print(len(Target_labels[0])) # should match the max_sequence_length

392
392
392


In [57]:
pn_history

'Stephanie madden is a 20 year old woman complaining of headache. Pain began yesterday morning and has been getting worse. Dull constant ache bilaterally, worse with walking, bending over, no alleviating factors. Did not respond to ibuprofen, Tylenol, or sleep. Complains of photobia, but not phonophobia, no aura. Subjective fever today. Neck stiffness. Nausea and vomiting. No dizzyness or lightheadedness, no weakness, no sick contacts.\r\nROS: otherwise negative\r\nPMH: none\r\nPSH: none\r\nHosp: none\r\nMed: Birth control pill\r\nAllergies: none\r\nSoc: works at sporting good store, lives with roomate, no tobacoo, etoh 2-3 drinks on weekends, marijuna 3-4 joints a week, sexually active and uses condoms'

In [58]:
np.array(input_id)

array([    0, 25093,  4134,   324,   475, 23004,    16,    10,   291,
          76,   793,   693, 13689,     9, 19344,     4, 23689,   880,
        2350,   662,     8,    34,    57,   562,  3007,     4, 38776,
        5891,  4285,   700,   741, 45931,     6,  3007,    19,  3051,
           6, 33842,    81,     6,   117, 32216, 16158,  2433,     4,
        6553,    45,  2519,     7, 34154,   658,  1001, 22132,     6,
         255,  4360,   225,  1168,     6,    50,  3581,     4, 20722,
        5069,     9, 17190, 33693,     6,    53,    45, 43676, 24938,
           6,   117, 34705,     4, 36994,  2088, 11696,   452,     4,
       33224, 37760,     4,   234, 17498,   102,     8, 23600,     4,
         440, 24719,   219,  1825,    50,  1109, 19279,  1825,     6,
         117,  8269,     6,   117,  4736,  9872,     4, 50121, 50118,
         500,  3196,    35,  3680,  2430, 50121, 50118,  5683,   725,
          35,  4146, 50121, 50118,  3888,   725,    35,  4146, 50121,
       50118,   725,

In [59]:
np.array(attn)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [60]:
type(np.array(input_id)[0])

numpy.int64

In [61]:
np.array(Target_label)

array([143, 143, 143, 143, 143, 143, 143, 143, 127, 127, 127, 143, 143,
       143, 143, 143, 143, 143, 128, 143, 143, 143, 143, 143, 143, 143,
       143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143,
       143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143,
       143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143,
       140, 140, 143, 143, 143, 143, 143, 143, 143, 143, 143, 142, 142,
       142, 143, 143, 131, 131, 143, 134, 134, 134, 143, 132, 143, 143,
       143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 141, 141,
       141, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143,
       143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143,
       143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143,
       143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 136,
       136, 136, 136, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143,
       143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 14

## Prepare tf datasets

In [62]:
print(len(input_id_sequence))
print(len(attn_sequence))
print(len(Target_labels))

1000
1000
1000


In [63]:
train_dataset = tf.data.Dataset.from_tensor_slices(((input_id_sequence, attn_sequence),Target_labels)) 

In [64]:
train_dataset

<_TensorSliceDataset element_spec=((TensorSpec(shape=(392,), dtype=tf.int32, name=None), TensorSpec(shape=(392,), dtype=tf.int32, name=None)), TensorSpec(shape=(392, 144), dtype=tf.uint8, name=None))>

Perform a 80-20 split into train and validation datasets 

In [65]:
train_size = len(input_id_sequence)
train_size

1000

In [66]:
train_size = len(input_id_sequence)
train_dataset = train_dataset.shuffle(1000)
ds_train = train_dataset.take(int(0.8*train_size))
ds_val = train_dataset.skip(int(0.8*train_size))

print(len(list(ds_train)), len(list(ds_val)))

800 200


In [67]:
ds_train

<_TakeDataset element_spec=((TensorSpec(shape=(392,), dtype=tf.int32, name=None), TensorSpec(shape=(392,), dtype=tf.int32, name=None)), TensorSpec(shape=(392, 144), dtype=tf.uint8, name=None))>

In [68]:
ds_val

<_SkipDataset element_spec=((TensorSpec(shape=(392,), dtype=tf.int32, name=None), TensorSpec(shape=(392,), dtype=tf.int32, name=None)), TensorSpec(shape=(392, 144), dtype=tf.uint8, name=None))>

In [69]:
batch_size = 5
ds_train = ds_train.batch(batch_size)
ds_val = ds_val.batch(batch_size)

In [70]:
len(list(ds_train.as_numpy_iterator()))

160

In [71]:
list(ds_train.as_numpy_iterator())[0][1]

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        ...,
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1]],

       [[0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        ...,
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1]],

       [[0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        ...,
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
   

In [72]:
list(ds_train.as_numpy_iterator())[0][0]

(array([[    0, 13123,   255, ...,     1,     1,     1],
        [    0,  1898, 25610, ...,     1,     1,     1],
        [    0,   725, 11337, ...,     1,     1,     1],
        [    0,   725, 11337, ...,     1,     1,     1],
        [    0,   725, 11337, ...,     1,     1,     1]], dtype=int32),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32))

In [73]:
len(list(ds_train.as_numpy_iterator())[0][0][0][0])

392

In [74]:
len(list(ds_train.as_numpy_iterator())[0][1])

5

In [75]:
len(list(ds_train.as_numpy_iterator())[0][1][0])

392

# Model training

In [76]:
SEED = 123
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [50]:
# review model architecture
# baseline model

In [77]:
# visualize model architecture
config = AutoConfig.from_pretrained(MODEL_NAME)
config

RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [78]:
len(input_id_sequence)

1000

In [79]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.9 MB/s eta 0:00:00


In [80]:
import tensorflow_addons as tfa

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [81]:
max_sequence_length

392

In [82]:
def build_model(config):
  # Input Layers:
  tokens = tf.keras.layers.Input(shape=(max_sequence_length,), name = 'tokens', dtype=tf.int32)
  attention = tf.keras.layers.Input(shape=(max_sequence_length,), name = 'attention', dtype=tf.int32)
  base_model = TFAutoModel.from_pretrained(MODEL_NAME, config=config)
  # Output Layers:
  out = base_model(tokens, attention_mask=attention)[0]
  #out = tf.keras.layers.Dropout(0.2)(out)
  out = tf.keras.layers.Dense(len(Label), activation='softmax')(out) 

  model = tf.keras.Model(inputs=(tokens, attention), outputs=out)
  #opt = keras.optimizers.Adam(learning_rate=0.00001)0.00005
  model.compile(
        optimizer=tf.keras.optimizers.Adam(5e-5),
        loss='categorical_crossentropy',
        #loss=tf.keras.losses.categorical_crossentropy,
        metrics=[tfa.metrics.F1Score(len(Label), average='micro')])
  return model

64 or 32? https://stackoverflow.com/questions/43816698/tensorflow-difference-in-performance-of-gather-for-int32-and-int64-indices-on-g

In [83]:
model = build_model(config)

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [84]:
for layer in model.layers:
  print(layer.output_shape)

[(None, 392)]
[(None, 392)]
TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=(None, 392, 768), pooler_output=(None, 768), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)
(None, 392, 144)


In [85]:
def fit_model(model, train_ds, val_ds, epochs=10):
    
    # Fits the model
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs
        )

    return model, history

In [86]:
model, history = fit_model(model, ds_train, ds_val, 10)

Epoch 1/10


InternalError: ignored

In [57]:
# def train_evaluate_model(train, val, model, epoch): 
#   """
#   Model training and evaluation
#   """
#   accuracy = []
#   loss = []
#   learning_curve_data = pd.DataFrame()

#   history = model.fit(train, validation_data=val, epochs=epoch)
#   #plot learning curve for train/val loss and accuracy
#   print('Training Learning Curve')
#   pd.DataFrame(history.history).plot(figsize = (15,10))
  

#   # evaluate model on test data 
#   accuracy.append(model.evaluate(test)[1])
#   loss.append(model.evaluate(test)[0])

#   # confusion matrix for each flower class
#   # predictions = np.array([]) 
#   # labels = np.array([])
#   # for x, y in val: #store predictions and labels in 1d array
#   #   predictions = np.concatenate([predictions, np.argmax(model.predict(x), axis = -1)])
#   #   labels = np.concatenate([labels, y])
#   # print('Confusion Matrix')
#   # disp = ConfusionMatrixDisplay.from_predictions(labels, 
#   #                                              predictions, 
#   #                                              display_labels = target_labels, 
#   #                                              normalize='true', 
#   #                                              cmap=plt.cm.Blues)

#   # print(classification_report(labels, predictions, target_names = target_labels))

#   return model, loss, accuracy


In [105]:
model, loss, accuracy = train_evaluate_model(ds_train, ds_val, model, 10)

NameError: ignored

In [ ]:
#model.save